In [10]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

import yfinance as yf

from datetime import datetime as dt

import plotly.graph_objects as go

### Creating the Training Data (create labels)

In [2]:
prices = yf.download('GOOG', start='2010-01-01', end=dt.today().strftime('%Y-%m-%d'))

[*********************100%***********************]  1 of 1 completed


In [3]:
prices.reset_index(inplace=True)
prices.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,15.615220,15.678981,15.547723,15.610239,15.610239,78541293
1,2010-01-05,15.620949,15.637387,15.480475,15.541497,15.541497,120638494
2,2010-01-06,15.588072,15.588072,15.102393,15.149715,15.149715,159744526
3,2010-01-07,15.178109,15.193053,14.760922,14.797037,14.797037,257533695
4,2010-01-08,14.744733,15.024933,14.672753,14.994298,14.994298,189680313


In [13]:
# extract the dates to use in the sliding window
dates = prices['Date']

# create windows of 15 days that jump every 5 days (5 days overlap)
windows = sliding_window_view(dates, window_shape = 15)[::5]

In [33]:
# create an array to store the associated following week returns for each window
associated_returns = np.zeros(len(windows)-1)

for i in range(len(windows)-1):
    
    # slice the dataframe
    window_data = prices.loc[prices['Date'].isin(windows[i])]
    
    # make the figure
    fig = go.Figure(data=[go.Candlestick(x=window_data['Date'], open=window_data['Open'], high=window_data['High'],
                                         low=window_data['Low'], close=window_data['Close'])])
    
    # remove uneccessary stuff from the figure
    fig.update_yaxes(showticklabels=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_layout(xaxis_rangeslider_visible=False)
    
    # save the figure
    fig.write_image(f'pattern_images/fig {i}.png')
    
    # find the associated next week return and write it to the associated_returns array
    next_window_data = prices.loc[prices['Date'].isin(windows[i+1])]
    next_window_data.reset_index(inplace=True, drop=True) # to have the index always from 0 to 9
    following_week_return = temp['Adj Close'].pct_change(periods=5)[5].round(3)
    associated_returns[i] = following_week_return

### Create labeled data for the CNN (stock pattern + its return for the following week)